In [1]:
import sys, pathlib
root = pathlib.Path.cwd().parents[0]
sys.path.append(str(root))

In [2]:
from typing import List
from pathlib import Path
import os, textwrap
from google import genai
from google.genai import types
from app.services.pptx_parser import PptxConverter
from app.services.schemas import AnalysisItem

In [ ]:
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [4]:
pptx_path = Path("C:/Users/kitano/RuleCheck/backend/tests/data/dummy_slide.pptx")
xml_str = PptxConverter.convert_to_xml(pptx_path, pretty=False)

In [5]:
RULES = textwrap.dedent(
"""
    1. 誤字脱字の確認と修正提案
    2. 製品名の不使用（成分名のみを使用）
    3. 恐怖をあおるような表現の禁止
    4. 治癒・完治など断定的治療効果表現の禁止
    5. 医療行為や医療現場を想起させる表現の禁止
    6. 効果・効能の誇大表現の禁止
    7. 出典や作成者名の明記確認（両方とも欠けている場合のみ指摘）
    8. 特定医療機関名への誘導表現の禁止
    9. 一般生活者向け広告表現の禁止（一般用医薬品等を除く）
"""
).strip()

In [6]:
prompt = f"""
あなたは医療・製薬・ヘルスケア資料を審査するコンプライアンス担当者です。
薬機法および以下の「チェック対象ルール」に照らし、違反または懸念箇所を指摘してください。

## チェック対象ルール
{RULES}

## スライド XML
{xml_str}

### 指摘カテゴリ
- 誤植 : 誤字脱字  
- 表現 : 表現規制・薬機法違反の可能性  
- 出典 : 出典および作成者情報が両方欠如  

### 出力要件
- JSON 配列のみを返す（テキスト装飾・説明文は禁止）  
- スキーマ: slideNumber, category, basis, issue, suggestion  
- slideNumber は XML の number 属性と一致させる  
- basis には違反根拠となるルール番号または薬機法条文を含める  
"""

In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0), # Disables thinking
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=list[AnalysisItem], 
    )
)

In [8]:
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_analysis_item: list[AnalysisItem] = response.parsed

[
  {
    "slideNumber": 1,
    "category": "誤植",
    "basis": "1",
    "issue": "「こんな事をる患者さんがよくいます」の「る」は誤字。",
    "suggestion": "「こんな事を言う患者さんがよくいます」に修正してください。"
  },
  {
    "slideNumber": 1,
    "category": "誤植",
    "basis": "1",
    "issue": "「五版を食べた後の」の「五版」は誤字。",
    "suggestion": "「ご飯を食べた後の」に修正してください。"
  },
  {
    "slideNumber": 1,
    "category": "表現",
    "basis": "2",
    "issue": "製品名「オングリザ」が使用されています。",
    "suggestion": "製品名ではなく成分名「サキサグリプチン水和物」を使用してください。"
  },
  {
    "slideNumber": 2,
    "category": "表現",
    "basis": "3, 4",
    "issue": "「糖尿病が絶対に発症してしまう」という表現は、恐怖をあおり、断定的な表現です。",
    "suggestion": "「糖尿病を発症するリスクが高まる可能性があります」など、恐怖をあおらず、断定を避けた表現に修正してください。"
  },
  {
    "slideNumber": 2,
    "category": "誤植",
    "basis": "1",
    "issue": "「疎外」は誤字。",
    "suggestion": "「阻害」に修正してください。"
  },
  {
    "slideNumber": 2,
    "category": "誤植",
    "basis": "1",
    "issue": "「消化ホモルン」は誤字。",
    "suggestion": "「消化ホルモン」に修正してください。"
  }
]


In [ ]:
import json
import pandas as pd

parsed_json = json.loads(response.text)
print(type(parsed_json))
print(parsed_json)

<class 'list'>
[{'slideNumber': 1, 'category': '誤植', 'basis': '1', 'issue': '「こんな事をる患者さんがよくいます」の「る」は誤字。', 'suggestion': '「こんな事を言う患者さんがよくいます」に修正してください。'}, {'slideNumber': 1, 'category': '誤植', 'basis': '1', 'issue': '「五版を食べた後の」の「五版」は誤字。', 'suggestion': '「ご飯を食べた後の」に修正してください。'}, {'slideNumber': 1, 'category': '表現', 'basis': '2', 'issue': '製品名「オングリザ」が使用されています。', 'suggestion': '製品名ではなく成分名「サキサグリプチン水和物」を使用してください。'}, {'slideNumber': 2, 'category': '表現', 'basis': '3, 4', 'issue': '「糖尿病が絶対に発症してしまう」という表現は、恐怖をあおり、断定的な表現です。', 'suggestion': '「糖尿病を発症するリスクが高まる可能性があります」など、恐怖をあおらず、断定を避けた表現に修正してください。'}, {'slideNumber': 2, 'category': '誤植', 'basis': '1', 'issue': '「疎外」は誤字。', 'suggestion': '「阻害」に修正してください。'}, {'slideNumber': 2, 'category': '誤植', 'basis': '1', 'issue': '「消化ホモルン」は誤字。', 'suggestion': '「消化ホルモン」に修正してください。'}]


In [21]:
import pandas as pd
df = pd.DataFrame(parsed_json)
df.head(10)

,slideNumber,category,basis,issue,suggestion
0,1,誤植,1,「こんな事をる患者さんがよくいます」の「る」は誤字。,「こんな事を言う患者さんがよくいます」に修正してください。
1,1,誤植,1,「五版を食べた後の」の「五版」は誤字。,「ご飯を食べた後の」に修正してください。
2,1,表現,2,製品名「オングリザ」が使用されています。,製品名ではなく成分名「サキサグリプチン水和物」を使用してください。
3,2,表現,"3, 4",「糖尿病が絶対に発症してしまう」という表現は、恐怖をあおり、断定的な表現です。,「糖尿病を発症するリスクが高まる可能性があります」など、恐怖をあおらず、断定を避けた表現に修...
4,2,誤植,1,「疎外」は誤字。,「阻害」に修正してください。
5,2,誤植,1,「消化ホモルン」は誤字。,「消化ホルモン」に修正してください。
